In [ ]:
# 

# Create target object and call it y

# Create X

# Split into validation and training data

# Specify Model

# Fit Model

# Make validation predictions and calculate mean absolute error

# Show a prediction result and real data.

### Importing libraries and Selecting Data for Modeling.

In [ ]:
# Importing the libraries
import pandas as pd
import numpy as np

In [ ]:
# Impoirting the dataset
house_prices_dataset_path = "./data/house-prices-advanced-regression-techniques/sample_submission.csv"
house_prices_test_path = "./data/house-prices-advanced-regression-techniques/test.csv"
house_prices_train_path = "./data/house-prices-advanced-regression-techniques/train.csv"
house_data_discription_path = "./data/house-prices-advanced-regression-techniques/data_description.txt"

house_prices = pd.read_csv(house_prices_dataset_path)
house_prices_test = pd.read_csv(house_prices_test_path)
house_prices_train = pd.read_csv(house_prices_test_path)

In [4]:
# Exporting the data (name of columns) to .xlsx file to more convenient work.
df = pd.DataFrame(house_prices_train.columns)
df.to_excel('./auxiliary_files/show_columns.xlsx', sheet_name='Column Names')

### Classes

In [5]:
class Nan_counter:
    """Counting NaN(true) and NaN(false) values"""  
    def __init__(self, data_set, col_name):
        self.data = data_set
        self.col_name = col_name
        
    def is_nan(self):
        """returning number of the NaN cells. In view True and False:
        True     1352
        False     107
        Name: column_name, dtype: int64"""
        return pd.isnull(self.data[self.col_name]).value_counts()

In [6]:
class Nan_columns:
    """
    - Finding the column(s) in dataframe that stored need number of NaN values,
    - Create a list with those columns"""
    def __init__(self, data_set, min_required_nan):
        self.data = data_set
        self.min_required_nan = min_required_nan
        
    def min_nan_number_in_column(self):
        """finding the columns that stroed NaN where each column stored less then 5 NaN values."""
        columns_of_dataset = self.data.columns.tolist()
        best_columns = []

        for i in columns_of_dataset:
            nan_sum = self.data[f"{i}"].isna().sum()            
            if nan_sum <= self.min_required_nan:
                best_columns.append(i)
        
        return best_columns

In [7]:
class Nan_string:
    """Show the string(s) that stored NaN value(s)"""
    def __init__(self, dataframe):
        self.df = dataframe
        
    def show_nan_string(self):
        """Return only those strings that contain NaN values"""
        # creating and filling a dictionary (#str: col_name)
        nan_str_col = dict([(x, self.df.columns[y]) for x, y in zip(*np.where(self.df.isna()))])
        
        # takes only keys from dictionary
        keys = list(nan_str_col.keys())

        return self.df.loc[keys]
        
    def show_nan_dict(self):
        """Return only number of the string and a name of the column that string(s) storing NaN values"""
        return dict([(x, self.df.columns[y]) for x, y in zip(*np.where(self.df.isna()))])

### Review Data

In [8]:
# reviewing all columns and find number NaN and not_a_Nan strings inside.
# for i in house_prices_test.columns:
#     test = Nan_counter(house_prices_test, f"{i}")
#     print(50 * "-")
#     print(test.is_nan())

In [9]:
# Counting the unique values inside column.
print(house_prices_test['LandContour'].value_counts(sort=False))

Bnk      54
HLS      70
Low      24
Lvl    1311
Name: LandContour, dtype: int64


In [10]:
# Select dataset columns which storing less than 3 NaN strings.
features = Nan_columns(house_prices_test, 3).min_nan_number_in_column()
print(features)

['Id', 'MSSubClass', 'LotArea', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'GarageCars', 'GarageArea', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SaleType', 'SaleCondition']


In [11]:
# Show str_num and col_name that storing NaN values.
show_nan_string = Nan_string(house_prices_test[features])
show_nan_string.show_nan_dict()

{95: 'KitchenQual',
 455: 'Utilities',
 485: 'Utilities',
 660: 'BsmtHalfBath',
 691: 'Exterior2nd',
 728: 'BsmtHalfBath',
 756: 'Functional',
 1013: 'Functional',
 1029: 'SaleType',
 1116: 'GarageArea'}

In [12]:
# Observing those strings that storing NaN values. String evaluation, visually analysis.
show_nan_string.show_nan_string()

,Id,MSSubClass,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
95,1556,50,10632,Pave,IR1,Lvl,AllPub,Inside,Gtl,ClearCr,...,0,248,0,0,0,0,1,2010,COD,Normal
455,1916,30,21780,Grvl,Reg,Lvl,NaN,Inside,Gtl,IDOTRR,...,24,0,0,0,0,0,3,2009,ConLD,Normal
485,1946,20,31220,Pave,IR1,Bnk,NaN,FR2,Gtl,Gilbert,...,0,144,0,0,0,750,5,2008,WD,Normal
660,2121,20,5940,Pave,IR1,Lvl,AllPub,FR3,Gtl,BrkSide,...,0,0,0,0,0,0,4,2008,ConLD,Abnorml
691,2152,30,19550,Pave,Reg,Lvl,AllPub,Inside,Gtl,Edwards,...,39,0,0,0,0,0,1,2008,WD,Normal
728,2189,20,47007,Pave,IR1,Lvl,AllPub,Inside,Gtl,Edwards,...,372,0,0,0,0,0,7,2008,WD,Normal
756,2217,20,14584,Pave,Reg,Low,AllPub,Inside,Mod,IDOTRR,...,0,0,0,0,0,0,2,2008,WD,Abnorml
1013,2474,50,10320,Pave,Reg,Lvl,AllPub,Corner,Gtl,IDOTRR,...,211,0,0,84,0,0,9,2007,COD,Abnorml
1029,2490,20,13770,Pave,Reg,Lvl,AllPub,Corner,Gtl,Sawyer,...,0,0,0,0,0,0,10,2007,NaN,Normal
1116,2577,70,9060,Pave,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,...,0,212,0,0,0,0,3,2007,WD,Alloca
